In [2]:
# Importing packages
import pandas as pd
import random
from sklearn.model_selection import train_test_split

# Importing Data

In [3]:
mon = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv')
tues = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv')
wed = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv')
thurs_morning = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
thurs_noon = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
fri_morning = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv')
fri_noon_ddos = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
fri_noon_portscan = pd.read_csv('../CIC-IDS-2017/CSVs/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
fri_noon_portscan.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
# concatenating all the days into a single dataset
days = [mon, tues, wed, thurs_morning, thurs_noon, fri_morning, fri_noon_ddos, fri_noon_portscan]
full_df = pd.concat(days)
full_df.shape

(2830743, 79)

Since the dataset is large, we'll randomly select 20% of the data to use so the classifiers can run within a reasonable amount of time so any bugs can be fixed, if necessary

In [5]:
# Randomly electing 20% of the whole dataset
random.seed(10)
df = full_df.sample(frac = 0.2)
df.shape

(566149, 79)

# Removing Redundant Features

In [6]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [7]:
df[[' Fwd Header Length', ' Fwd Header Length.1']]

,Fwd Header Length,Fwd Header Length.1
297121,20,20
194875,40,40
138786,20,20
489999,80,80
589597,64,64
...,...,...
23093,40,40
110930,20,20
7437,160,160
152934,328,328


From the list of column names above, we can see that there are two columns named 'Fwd Header Length' columns and they are exactly the same. We will remove the ' Fwd Header Length' column to decrease the number of features that will be processed in the future tasks. 

In [8]:
# Removing the redundant feature
df = df.drop(' Fwd Header Length.1', axis = 1)
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

# Relabeling Attack Classes

In [9]:
df[' Label'].value_counts()

BENIGN                        454566
DoS Hulk                       46009
PortScan                       31952
DDoS                           25722
DoS GoldenEye                   2004
FTP-Patator                     1577
SSH-Patator                     1231
DoS slowloris                   1147
DoS Slowhttptest                1118
Bot                              382
Web Attack - Brute Force         302
Web Attack - XSS                 123
Infiltration                       8
Web Attack - Sql Injection         8
Name:  Label, dtype: int64

Based on the paper, Kurniabudi, et al. relabeled the attack classes as: 

- Normal = [Benign]
- Bot = [Bot]
- Brute Force = [FTP-Patator, SSH-Patator]
- Dos/Ddos = [DDos, DoS, DoS GoldenEye, DoS Hulk, DoS Slowhttptest, Dos slowloris, Heartbleed]
- Infiltration = [Infiltration]
- Portscan = [PortScan]
- Web Attack = [Web Attack-Brute Force, Web Attack-Sql Injection, Web Attack-XSS]

So we will do the same in the following. 


In [10]:
df[' Label'].unique()

array(['BENIGN', 'PortScan', 'DoS Hulk', 'DDoS', 'FTP-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'SSH-Patator',
       'DoS GoldenEye', 'Bot', 'Web Attack - Brute Force',
       'Web Attack - XSS', 'Infiltration', 'Web Attack - Sql Injection'],
      dtype=object)

In [11]:
label_dict = {'BENIGN': 'Normal', 
              'FTP-Patator': 'Brute Force',
              'SSH-Patator': 'Brute Force',
              'DDoS': 'DoS/DdoS', 
              'DoS': 'DoS/DdoS',
              'DoS GoldenEye': 'DoS/DdoS',
              'DoS Hulk': 'DoS/DdoS',
              'DoS Slowhttptest': 'DoS/DdoS',
              'DoS slowloris': 'DoS/DdoS',
              'Heartbleed': 'DoS/DdoS',
              'PortScan': 'Portscan',
              'Bot': 'Bot',
              'Infiltration': 'Infiltration',
              'Web Attack - Brute Force': 'Web Attack',
              'Web Attack - Sql Injection': 'Web Attack',
              'Web Attack - XSS': 'Web Attack'}
df[' Label'] = df[' Label'].apply(lambda x: label_dict[x])
df[' Label'].unique()

array(['Normal', 'Portscan', 'DoS/DdoS', 'Brute Force', 'Bot',
       'Web Attack', 'Infiltration'], dtype=object)

# Splitting Data into Train and Test Sets

In [12]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.70, random_state = 0)

In [13]:
print(X_train.shape)
print(X_test.shape)

(396304, 77)
(169845, 77)


# Saving Preprocessed Data into CSV Files

In [14]:
# Combining train and test info into their respective data sets 
X_train['Label'] = y_train
X_test['Label'] = y_test

In [15]:
# Exporting train and test datasets into CSV files so it can be used in the feature selection step
X_train.to_csv('../preprocess_data/train.csv', na_rep = 'NA')
X_test.to_csv('../preprocess_data/test.csv', na_rep = 'NA')